In [1]:
!pip uninstall -y kfp

Found existing installation: kfp 1.4.0
Uninstalling kfp-1.4.0:
  Successfully uninstalled kfp-1.4.0


In [19]:
!pip install kfp pytorch_lightning==1.0.2

     |████████████████████████████████| 532 kB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 27.7 MB/s eta 0:00:01
     |██████████████████████████████▎ | 760.7 MB 164.3 MB/s eta 0:00:012 MB 100.7 MB/s eta 0:00:08.8 MB 100.7 MB/s eta 0:00:08              | 168.1 MB 190.0 MB/s eta 0:00:04     |████████▋                       | 216.7 MB 190.0 MB/s eta 0:00:04     |█████████▌                      | 239.0 MB 190.0 MB/s eta 0:00:03     |██████████████████              | 454.5 MB 169.5 MB/s eta 0:00:03��███████▋     | 667.4 MB 162.0 MB/s eta 0:00:01     |█████████████████████████████▌  | 740.5 MB 164.3 MB/s eta 0:00:01

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 804.1 MB 168.0 MB/s eta 0:00:01

In [29]:
import kfp

from kfp.components import load_component_from_file, load_component_from_url, create_component_from_func
from kfp import dsl
from kfp import compiler
import json

In [30]:
kfp.__version__

'1.4.0'

In [31]:
#Update values for the load balancer and auth session
HOST='istio-ingressgateway.istio-system.svc.cluster.local'
AUTH="authservice_session=MTYxOTYwMDA3MXxOd3dBTkRaTlFVRlJSazVLTkV0T1ZVcFZRamRYVWs5S1ZrRlJTVVZFTWxWQlVEWkhSbEJJU1ZKSFJsUlpVMFZGVGpSS05VVmFSVkU9fEc65xAZpii1Q9jqI0khMApPA-349QLI5lt2siZOD-AF"
namespace="kubeflow-user-example-com"

In [32]:
client = kfp.Client(host=HOST+"/pipeline", cookies=AUTH)
experiments = client.list_experiments(namespace="kubeflow-user-example-com")
my_experiment = experiments.experiments[0]
my_experiment

{'created_at': datetime.datetime(2021, 4, 22, 8, 44, 39, tzinfo=tzlocal()),
 'description': None,
 'id': 'aac96a63-616e-4d88-9334-6ca8df2bb956',
 'name': 'Default',
 'resource_references': [{'key': {'id': 'kubeflow-user-example-com',
                                  'type': 'NAMESPACE'},
                          'name': None,
                          'relationship': 'OWNER'}],
 'storage_state': 'STORAGESTATE_AVAILABLE'}

In [78]:
prepare_tensorboard = load_component_from_file(
    os.path.join(
        os.path.dirname('test.ipynb'),
        '../../../components/tensorflow/tensorboard/prepare_tensorboard/component.yaml'
    )
)
# New features used locally not released yet, so we can only use local import.
# prepare_tensorboard = components.load_component_from_url(
#     'https://raw.githubusercontent.com/kubeflow/pipelines/1.5.0/components/tensorflow/tensorboard/prepare_tensorboard/component.yaml'
# )


def train(minio_endpoint: 'URI', log_bucket: str, log_dir: 'Path'):
    import os
    import shutil
    from pathlib import Path

    import pytorch_lightning as pl
    import torch
    from minio import Minio
    from pytorch_lightning.loggers import TensorBoardLogger
    from pytorch_lightning.metrics.functional import accuracy
    from torch import nn
    from torch.nn import functional as F
    from torch.utils.data import DataLoader, random_split
    from torchvision import transforms
    from torchvision.datasets import MNIST

    class LitMNIST(pl.LightningModule):

        def __init__(self, data_dir='./', hidden_size=64, learning_rate=2e-4):

            super().__init__()

            # Set our init args as class attributes
            self.data_dir = data_dir
            self.hidden_size = hidden_size
            self.learning_rate = learning_rate

            # Hardcode some dataset specific attributes
            self.num_classes = 10
            self.dims = (1, 28, 28)
            channels, width, height = self.dims
            self.transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ])

            # Define PyTorch model
            self.model = nn.Sequential(
                nn.Flatten(),
                nn.Linear(channels * width * height, hidden_size),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(hidden_size, hidden_size),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(hidden_size, self.num_classes)
            )

        def forward(self, x):
            x = self.model(x)
            return F.log_softmax(x, dim=1)

        def training_step(self, batch, batch_idx):
            x, y = batch
            logits = self(x)
            loss = F.nll_loss(logits, y)
            return loss

        def validation_step(self, batch, batch_idx):
            x, y = batch
            logits = self(x)
            loss = F.nll_loss(logits, y)
            preds = torch.argmax(logits, dim=1)
            acc = accuracy(preds, y)

            # Calling self.log will surface up scalars for you in TensorBoard
            self.log('val_loss', loss, prog_bar=True)
            self.log('val_acc', acc, prog_bar=True)
            return loss

        def test_step(self, batch, batch_idx):
            # Here we just reuse the validation_step for testing
            return self.validation_step(batch, batch_idx)

        def configure_optimizers(self):
            optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
            return optimizer

        def prepare_data(self):
            # download
            MNIST(self.data_dir, train=True, download=True)
            MNIST(self.data_dir, train=False, download=True)

        def setup(self, stage=None):

            # Assign train/val datasets for use in dataloaders
            if stage == 'fit' or stage is None:
                mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
                self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

            # Assign test dataset for use in dataloader(s)
            if stage == 'test' or stage is None:
                self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)

        def train_dataloader(self):
            return DataLoader(self.mnist_train, batch_size=32)

        def val_dataloader(self):
            return DataLoader(self.mnist_val, batch_size=32)

        def test_dataloader(self):
            return DataLoader(self.mnist_test, batch_size=32)


    model = LitMNIST()

    tensorboard_root = os.getcwd()

    if os.path.exists(os.path.join(tensorboard_root, "mnist_lightning_kubeflow")):
        shutil.rmtree(os.path.join(tensorboard_root, "mnist_lightning_kubeflow"))

    Path(tensorboard_root).mkdir(parents=True, exist_ok=True)
    # Tensorboard root name of the logging directory
    tboard = TensorBoardLogger(tensorboard_root, "mnist_lightning_kubeflow")

    trainer = pl.Trainer(max_epochs=1, logger=tboard)
    trainer.fit(model)
    trainer.test()


    ###########################
    # USING MINIO #
    ###########################


    client = Minio(
        minio_endpoint, access_key="minio", secret_key="minio123", secure=False
    )

    bucket_name = log_bucket
    folder_name = 'mnist'
    
    for path in Path("mnist_lightning_kubeflow/version_0/").rglob("*"):
        if not path.is_dir():
            client.fput_object(
                bucket_name=bucket_name,
                object_name=os.path.join(
                    folder_name, os.path.relpath(start='mnist_lightning_kubeflow/version_0', path=path)
                ),
                file_path=path,
            )

# tensorflow/tensorflow:2.4 may fail with image pull backoff, because of dockerhub rate limiting.
train_op = create_component_from_func(
    train,
    base_image='pytorch/pytorch:latest',
    packages_to_install=['minio', 'pytorch_lightning', 'torchvision'],  # TODO: pin minio version
)


In [79]:
@dsl.pipeline(name='pipeline-tensorboard-minio')
def my_pipeline(
    minio_endpoint='minio-service.kubeflow:9000',
    log_bucket='mlpipeline',
#     log_dir=f'tensorboard/logs/{dsl.RUN_ID_PLACEHOLDER}',
    log_dir='mnist',
    # Pin to tensorflow 2.3, because in 2.4+ tensorboard cannot load in KFP:
    # refer to https://github.com/kubeflow/pipelines/issues/5521.
    tf_image='gcr.io/deeplearning-platform-release/tf2-cpu.2-3:latest'
):
    # tensorboard uses s3 protocol to access minio
    prepare_tb_task = prepare_tensorboard(
        log_dir_uri=f's3://{log_bucket}/{log_dir}',
        image=tf_image,
        pod_template_spec=json.dumps({
            'spec': {
                'containers': [{
                    # These env vars make tensorboard access KFP in-cluster minio
                    # using s3 protocol.
                    # Reference: https://blog.min.io/hyper-scale-machine-learning-with-minio-and-tensorflow/
                    'env': [{
                        'name': 'AWS_ACCESS_KEY_ID',
                        'valueFrom': {
                            'secretKeyRef': {
                                'name': 'mlpipeline-minio-artifact',
                                'key': 'accesskey'
                            }
                        }
                    }, {
                        'name': 'AWS_SECRET_ACCESS_KEY',
                        'valueFrom': {
                            'secretKeyRef': {
                                'name': 'mlpipeline-minio-artifact',
                                'key': 'secretkey'
                            }
                        }
                    }, {
                        'name': 'AWS_REGION',
                        'value': 'minio'
                    }, {
                        'name': 'S3_ENDPOINT',
                        'value': f'{minio_endpoint}',
                    }, {
                        'name': 'S3_USE_HTTPS',
                        'value': '0',
                    }, {
                        'name': 'S3_VERIFY_SSL',
                        'value': '0',
                    }]
                }],
            },
        })
    )
    train_task = train_op(
        minio_endpoint=minio_endpoint,
        log_bucket=log_bucket,
        log_dir=log_dir,
    )
    # optional, let training task use the same tensorflow image as specified tensorboard
    train_task.container.image = tf_image
    train_task.after(prepare_tb_task)


In [80]:
compiler.Compiler().compile(my_pipeline, 'pytorch.tar.gz', type_check=True)

In [81]:
run = client.run_pipeline(my_experiment.id, 'pytorch-model', 'pytorch.tar.gz')